In [1]:
import numpy as np
import cv2

In [2]:
with_mask = np.load('with_mask.npy')
without_mask = np.load('without_mask.npy')

In [3]:
with_mask.shape

(10, 50, 50, 3)

In [4]:
without_mask.shape

(10, 50, 50, 3)

In [6]:
with_mask = with_mask.reshape(10,50 *50 * 3)
without_mask = without_mask.reshape(10,50 *50 * 3)

In [7]:
with_mask.shape

(10, 7500)

In [8]:
without_mask.shape

(10, 7500)

In [9]:
x = np.r_[with_mask , without_mask]

In [10]:
x.shape

(20, 7500)

In [11]:
lables = np.zeros(x.shape[0])

In [12]:
lables[200:] = 1.0

In [13]:
names = {0: 'mask',1 : 'no mask'}

In [14]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
x_train, x_test, ytrain, y_test = train_test_split(x,lables, test_size= 0.25)

In [17]:
x_train.shape

(15, 7500)

In [18]:
from sklearn.decomposition import PCA

In [19]:
pca = PCA(n_components = 3)
x_train = pca.fit_transform(x_train)

In [20]:
x_train[0]

array([-828.11274206, -202.62601336, -318.88879738])

In [21]:
x_train.shape

(15, 3)

In [22]:
svm = SVC()
svm.fit(x_train, ytrain)

ValueError: The number of classes has to be greater than one; got 1 class

In [ ]:
x_test = pca.transform(x_test)
y_pred = svm.predict(x_test)


In [ ]:
accuracy_score(y_test , y_pred)

In [8]:
cascade_file = r'C:\Face-Mask-Detection-master\haarcascade_frontalface_default.xml'
haar_data = cv2.CascadeClassifier(cascade_file)
capture = cv2.VideoCapture(0)  # Change index to 0 for default camera
data = []
data_without_mask =[]
font = cv2.FONT_HERSHEY_COMPLEX
while True:
    flag, img = capture.read()
    if flag:
        faces = haar_data.detectMultiScale(img)
        # Check if any faces are detected
        if len(faces) > 0:
            # Draw rectangles around detected faces
            for x, y, w, h in faces:
                cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 255), 4)
                face= img[y:y+h, x:x+w, :]
                face = cv2.resize(face, (50,50))
                face = face.reshape(1,  -1)
                face = pca.transform(face)
                pred = svm.predict(face) 
                n = names[int(pred)]
                cv2.putText(img, n, (x,y), font, 1, (244, 250, 250), 4)
                print(n)
            # Display the result
            cv2.imshow('result', img)
            
            # Check if the window is closed or ESC key is pressed
            key = cv2.waitKey(100)  # Increase delay to 100 milliseconds
            if key == 27:  # ESC key
                break
    else:
        print("Error: Unable to capture frame from camera")
        break

capture.release()
cv2.destroyAllWindows()


NameError: name 'pca' is not defined